### Check if you are able to get response from Claude through AWS Bedrock

In [14]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_claude",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

Hi!


In [15]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat_titan",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


Hello!


### Check if the API docs are fetched correctly

In [ ]:
import requests
weather_response = requests.get("http://localhost:8000/weather")

print(weather_response.json()["response"])

### Check if the user query is fetched correctly

In [2]:
import requests
weather_response = requests.post("http://localhost:8000/user_query", json={"user_query": "What is the weather in London?"})

print(weather_response.json())

{'response': 'What is the weather in London?'}


### Test the query with Claude as both the mother as well as the child agent

In [43]:
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "What is the weather like at latitude 37.7749 and longitude -122.4194"})
print(query_response.json())

agent_response = requests.get("http://localhost:8000/claude_child_agent", 
                            params={"user_query": query_response.json()["response"]})

{'response': 'What is the weather like at latitude 37.7749 and longitude -122.4194'}


### Test the query with Claude as the mother agent and Titan as the child agent

In [52]:
query_response = requests.post("http://localhost:8000/user_query", 
                             json={"user_query": "How is the humidity in Saulo Paulo Like?"})
print("Query Response:", query_response.json())

agent_response = requests.get("http://localhost:8000/titan_child_agent", 
                            params={"user_query": query_response.json()["response"]})
print("Agent Response:", agent_response.json())

Query Response: {'response': 'How is the humidity in Saulo Paulo Like?'}
Agent Response: {'message': 'Child agent successfully created', 'response': 'import boto3\nimport requests\nimport json\nimport os\nfrom bedrock_handler import call_titan\nfrom dotenv import load_dotenv\n\nload_dotenv()\n\napi_key = os.getenv("OPENWEATHERMAP_API_KEY")\n\ndef get_data(city):\n    try:\n        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"\n        response = requests.get(url)\n        response.raise_for_status()\n        data = response.json()\n        temp = data["main"]["temp"]\n        description = data["weather"][0]["description"]\n        output = f"Temperature: {temp} C, Weather: {description}"\n        return output\n    except requests.exceptions.RequestException as e:\n        return f"Error: {e}"\n    except (KeyError, IndexError):\n        return "Error: Unable to extract weather data from API response"\n    except Exception as e:\n      

### Added a route to test and verify the health of the child agent code and execute it to get the response from the child agent

In [53]:
import requests

# The path to the agent file you want to verify
agent_file = "child_agent.py"

# Make the POST request
response = requests.post(f"http://localhost:8000/verify_agent?agent_file={agent_file}")

# Print the response
print(response.json())

{'status': 'success', 'syntax_valid': True, 'message': 'Agent file executed successfully', 'output': ''}


In [ ]:
import child_agent
!python child_agent.py